# What apps are most liked by users

- This goal of this project is to analyze what type of apps attract more users.

We will use two datasets.
1.[Google play app data](https://www.kaggle.com/lava18/google-play-store-apps) 
2.[Apple Mobile App Data](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

In [1]:
from csv import reader

def explore_data(dataset,start,end,rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows:',len(dataset))
        print('Number of columns:', len(dataset[0]))
        print('\n')
    
        
def open_data(file_name):
    file = open(file_name,encoding='utf8')
    read_file = reader(file)
    dataset = list(read_file)
    return dataset
    
ios_file=open_data('AppleStore.csv')
google_file=open_data('googleplaystore.csv')

In [2]:
#explore_data(ios_file[1:],1,2,True)
explore_data(google_file[1:],1,2,True)

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13




In [3]:
print(google_file[0:2])

[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']]


In [4]:
print(google_file[10473])
print(len(google_file[10473]))

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
12


In [5]:
del google_file[10473]
print(google_file[10473])
print(len(google_file[10473]))


['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']
13


# found duplicate data from disscussion 
From the [disscussion](https://www.kaggle.com/lava18/google-play-store-apps/discussion) we can see that there may be duplicate datas. 
So I am going to make a function which finds out duplicate data entries. 

1.First I am going to find what kind of data has duplicate entries
- i will try to find apps with dupliactes entries

In [6]:
print(google_file[0])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [7]:
def find_duplicates(dataset):
    duplicate = []
    unique_app = []
    for row in dataset[1:]:
        if row[0] in unique_app:
            duplicate.append(row[0])
        else:
            unique_app.append(row[0])
    
    print('Number of duplicate app: ',len(duplicate))
    print('Example of duplicate app:', duplicate[:15])
    return unique_app,duplicate
            

Here we will first get all the duplicates app by the `find_duplicates` function
the function return alist the name of the app that has duplicate entries.

-> `find dupliactes` has two parameters. dataset and header 
-> if your dataset doesnt have header row, pass `header=False` as parameter. 
->The default value is `True` as we assume that the dataset has header while being passed as arguement 

In [8]:
android_unique_apps , android_duplicate = find_duplicates(google_file)

Number of duplicate app:  1181
Example of duplicate app: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


-> the `find dupliactes` return two values. `unique apps` and apps that have `duplicate values` in the dataset

# Cleaning out duplicates

- listing the unique apps with their maximum ratings


- we can see here that the rating are in 3rd colomn
- ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']

- We have a list `android_unique_apps` containg the name of unique app . we will use this same technique to extract out the unique apps' maximum rating.



In [9]:
unique_app_max_rating={}
for row in google_file[1:]:
    name = row[0]
    rating = float(row[3])
    if name in unique_app_max_rating:
        unique_app_max_rating[name]=max(unique_app_max_rating[name],rating)
    else:
        unique_app_max_rating[name]=rating
                                                                        

We have seen there is 1181 duplicate entries in the dataset. 

In [10]:
print(len(google_file[1:]))

10840


We can see that after deleting the duplicate entries , we have exact `9659` values. 

In [11]:
print(unique_app_max_rating['Instagram'])

66577446.0


- `we'd like to analyze only the apps that are directed toward an English-speaking audience. However, if we explore the data long enough, we'll find that both data sets have apps with names that suggest they are not directed toward an English-speaking audience.`


- At first we counted the unique apps with their max ratings.

- Then we looped throug the data and check if the row contains the maximum rating 

- we will also keep count of the already visited apps so that duplicate entries with max ratings also get eliminated

In [12]:
android_header = google_file[0]
android_clean = []
already_added = []
for row in google_file[1:]:
    name=row[0]
    rating=float(row[3])
    max_rating=unique_app_max_rating[name]
    if (rating==max_rating) and (name not in already_added):
        #(row)
        android_clean.append(row)
        already_added.append(name)
        
#print('sesh')
print(len(android_clean))

9659


`android_clean` is the clean data without duplicates.

# Remiving NON English App

We're not interested in keeping these apps, so we'll remove them. One way to go about this is to remove each app with a name containing a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;), and other symbols (+, *, /).

Behind the scenes, each character we use in a string has a corresponding number associated with it. For instance, the corresponding number for character 'a' is 97, character 'A' is 65, and character '爱' is 29,233. We can get the corresponding number of each character using the ord() built-in function.

In [13]:
print(ord('a'))
print(ord('z'))
print(ord('A'))
print(ord('Z'))

97
122
65
90


- part 1
  - we will keep names with ascii values
     - we have 128 ascii characters indexed from 0 to 127

In [14]:
def is_english(name):
    for letter in name:
        nvalue = ord(letter)
        if nvalue> 127:
            return False
    return True           

This function works just fine. But its wont work with all the english names bacause some english apps may use special characters such as
- `Docs To Go™ Free Office Suite` returns it as a non english name.

In [15]:
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

False
False


- part 2
    - If we're going to use the function we've created, we'll lose useful data since many English apps will be incorrectly labeled as non-English. To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English. Our filter function is still not perfect, but it should be fairly effective.

In [16]:
def is_english(name):
    cnt = 0
    for letter in name:
        nvalue = ord(letter)
        if nvalue> 127:
            cnt +=1
    if cnt>3:
        return False
    return True   

print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
True


# removing non english app from android

In [17]:
print(android_header)

android_english=[]
for row in android_clean:
    name=row[0]
    if is_english(name)==True:
        android_english.append(row)


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [18]:
explore_data(android_english,1,3,True)

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13




# removing non english apps from ios 

In [19]:
print(len(ios_file))
ios_header = ios_file[0]
ios_data = ios_file[1:]
print(ios_header)

ios_english=[]
for row in ios_data:
    name=row[1]
    if is_english(name)==True:
        ios_english.append(row)
        
explore_data(ios_english,1,3,True)

7198
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 6183
Number of columns: 16




We can see that we're left with `9614 Android apps` and `6183 iOS apps`.

In [20]:
print(android_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [21]:
print(android_english[0])
print(android_english[0][7])

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
0


# Removing non free app
- part 1
    - android data has price feature in 8th colun (7th index) 
    - we will remove any app which has price > 0

In [22]:
free_android = []
for row in android_english:
    price = row[7]
    if price=='0':
        free_android.append(row)

explore_data(free_android,1,3,True)

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 8864
Number of columns: 13




In [23]:
free_ios = []
for row in ios_english:
    price=row[4]
    if price=='0.0':
        free_ios.append(row)
        
explore_data(free_ios,1,3,True)
        

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 3222
Number of columns: 16




We're left with `8864 Android apps` and `3222 iOS apps`, which should be enough for our analysis.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

    - Build a minimal Android version of the app, and add it to Google Play.
    - If the app has a good response from users, we develop it further.
    - If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

part 1
    - finding genre freequencies on android

In [24]:
android_genre = {}

for row in free_android:
    genre = row[9]
    if genre in android_genre:
        android_genre[genre]+=1
    else:
        android_genre[genre]=1

fav_android_genre=''
max_android_freq=0

for key,freq in android_genre.items():
    if max_android_freq<freq:
        max_android_freq=freq
        fav_android_genre=key
        
print(fav_android_genre)
print(max_android_freq)

Tools
749


In [25]:
ios_genre = {}

for row in free_ios:
    genre = row[11]
    if genre in ios_genre:
        ios_genre[genre]+=1
    else:
        ios_genre[genre]=1

fav_ios_genre=''
max_ios_freq=0

for key,freq in ios_genre.items():
    if max_ios_freq<freq:
        max_ios_freq=freq
        fav_ios_genre = key
        
print(fav_ios_genre)
print(max_ios_freq)

Games
1874


In [26]:
print(android_header,'\n',ios_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 
 ['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


we'll need to build a frequency table for the prime_genre column of the App Store data set, and for the Genres and Category columns of the Google Play data set.

We'll build two functions we can use to analyze the frequency tables:

   - One function to generate frequency tables that show percentages
   - Another function we can use to display the percentages in a descending order


In [27]:
def freq_table(dataset,index):
    freq_dic = {}
    for row in dataset:
        feature = row[index]
        if feature in freq_dic:
            freq_dic[feature]+=1
        else:
            freq_dic[feature]=1
    total = len(dataset)
    freq_percentage =  {}
    for key in freq_dic:
        percentage = (freq_dic[key]/total)*100
        freq_percentage[key] = percentage 
    
    return freq_percentage

       
def display_table(dataset, index):
    table =  freq_table(dataset,index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key],key)
        table_display.append(key_val_as_tuple)
    
    table_sorted = sorted(table_display, reverse=True)
    for entry in table_sorted:
        print(entry[1],':',entry[0])


# ios popular genre analysis

getting percentage table for ios apps, feature = prime_genre , index = 11 (0 based indexing)

In [28]:
display_table(free_ios,11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


- We can see that more than 50% free english app are of Games. -
- Entertainment has the second heighest , then photo and video 
- Educational apps has like 3.662321% apps and 
- Social networking apps has almost equal percentage as the socal networking
 
 In ios , we can conclide that entertainment related genres are more popular than tools. 

# android popular genre analysis
part 1

getting percentage table for android apps, feature = genre , index = 9 (o based indexing)

In [29]:
display_table(free_android,9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

Here genre has many types, we will get back to here later. 

Part 2
    - getting percentage table for android apps,feature = category  , index = 1 (0 based indexing)

In [30]:
display_table(free_android,1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

we can see that `Family` is the most popular category here , followed by `Game` category. 
 
 - If we go to the `google play` page, we will see that in `Family` category:
   ![image](imagename.png "Title") 
   - there is mostly games and apps for kids so entertainment is also a popular genre among free english android app



 - the second popuar genre  is games 
 - TOOLS : 8.461191335740072
 - BUSINESS : 4.591606498194946
 - LIFESTYLE : 3.9034296028880866
 - PRODUCTIVITY : 3.892148014440433
 - FINANCE : 3.7003610108303246
 - MEDICAL : 3.531137184115524
 - SPORTS : 3.395758122743682
 

We can see that in android data, along with entertaining apps, tools and educational apps have good popularity


# Analyzing the data (continue.)

The frequency tables we analyzed on the previous screen showed us that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and fun apps. Now, we'd like to get an idea about the kind of apps with the most users.

# Calculate the average number of installs for each app genre
One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but this information is missing for the App Store data set. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

# popular genre acourding to number of installs(ios)

In [44]:
unique_ios_apps = freq_table(free_ios,-5)

table_display = [] 

for genre in unique_ios_apps:
    total=0
    len_genre=0
    for app in free_ios:
        genre_app=app[-5]
        if genre_app==genre:
            user_rating = float(app[5])
            total+=user_rating
            len_genre+=1
    avg_genre_rating = total/len_genre
    
    key_val_as_tuple = (avg_genre_rating,genre)
    table_display.append(key_val_as_tuple)
        
table_sorted = sorted(table_display, reverse=True)
for entry in table_sorted:
    print(entry[1],':',entry[0])
            


Navigation : 86090.33333333333
Reference : 74942.11111111111
Social Networking : 71548.34905660378
Music : 57326.530303030304
Weather : 52279.892857142855
Book : 39758.5
Food & Drink : 33333.92307692308
Finance : 31467.944444444445
Photo & Video : 28441.54375
Travel : 28243.8
Shopping : 26919.690476190477
Health & Fitness : 23298.015384615384
Sports : 23008.898550724636
Games : 22788.6696905016
News : 21248.023255813954
Productivity : 21028.410714285714
Utilities : 18684.456790123455
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Business : 7491.117647058823
Education : 7003.983050847458
Catalogs : 4004.0
Medical : 612.0


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [32]:
for app in free_ios:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5])
        
        

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


In [47]:
for app in free_ios:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


In the previous screen, we came up with an app profile recommendation for the App Store based on the number of user ratings. We have data about the number of installs for the Google Play market, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.)

# popular genre acourding to number of installs(android) 

In [33]:
display_table(free_android,5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343




One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to float — this means that 
we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).


In [46]:
unique_android_apps=freq_table(free_android,1)

table_display = []
for category in unique_android_apps:
    total=0
    len_category=0
    for app in free_android:
        category_app=app[1]
        if category_app==category:
            installs = app[5]
            installs = installs.replace(',','')
            installs = installs.replace('+','')
            installs = float(installs)
            total+=installs
            len_category+=1
    avg_category_rating = total/len_category
    
    key_val_as_tuple = (avg_category_rating,category)
    table_display.append(key_val_as_tuple)
        
table_sorted = sorted(table_display, reverse=True)
for entry in table_sorted:
    print(entry[1],':',entry[0])

COMMUNICATION : 38456119.167247385
VIDEO_PLAYERS : 24727872.452830188
SOCIAL : 23253652.127118643
PHOTOGRAPHY : 17840110.40229885
PRODUCTIVITY : 16787331.344927534
GAME : 15588015.603248259
TRAVEL_AND_LOCAL : 13984077.710144928
ENTERTAINMENT : 11640705.88235294
TOOLS : 10801391.298666667
NEWS_AND_MAGAZINES : 9549178.467741935
BOOKS_AND_REFERENCE : 8767811.894736841
SHOPPING : 7036877.311557789
PERSONALIZATION : 5201482.6122448975
WEATHER : 5074486.197183099
HEALTH_AND_FITNESS : 4188821.9853479853
MAPS_AND_NAVIGATION : 4056941.7741935486
FAMILY : 3695641.8198090694
SPORTS : 3638640.1428571427
ART_AND_DESIGN : 1986335.0877192982
FOOD_AND_DRINK : 1924897.7363636363
EDUCATION : 1833495.145631068
BUSINESS : 1712290.1474201474
LIFESTYLE : 1437816.2687861272
FINANCE : 1387692.475609756
HOUSE_AND_HOME : 1331540.5616438356
DATING : 854028.8303030303
COMICS : 817657.2727272727
AUTO_AND_VEHICLES : 647317.8170731707
LIBRARIES_AND_DEMO : 638503.734939759
PARENTING : 542603.6206896552
BEAUTY : 51315

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:


In [50]:
for app in free_android:
    if app[1]=='COMMUNICATION':
        installs = app[5]
        installs = installs.replace(',','')
        installs = installs.replace('+','')
        installs = float(installs)
        print(app[0],':',installs)
        

WhatsApp Messenger : 1000000000.0
Messenger for SMS : 10000000.0
My Tele2 : 5000000.0
imo beta free calls and text : 100000000.0
Contacts : 50000000.0
Call Free – Free Call : 5000000.0
Web Browser & Explorer : 5000000.0
Browser 4G : 10000000.0
MegaFon Dashboard : 10000000.0
ZenUI Dialer & Contacts : 10000000.0
Cricket Visual Voicemail : 10000000.0
TracFone My Account : 1000000.0
Xperia Link™ : 10000000.0
TouchPal Keyboard - Fun Emoji & Android Keyboard : 10000000.0
Skype Lite - Free Video Call & Chat : 5000000.0
My magenta : 1000000.0
Android Messages : 100000000.0
Google Duo - High Quality Video Calls : 500000000.0
Seznam.cz : 1000000.0
Antillean Gold Telegram (original version) : 100000.0
AT&T Visual Voicemail : 10000000.0
GMX Mail : 10000000.0
Omlet Chat : 10000000.0
My Vodacom SA : 5000000.0
Microsoft Edge : 5000000.0
Messenger – Text and Video Chat for Free : 1000000000.0
imo free video calls and chat : 500000000.0
Calls & Text by Mo+ : 5000000.0
free video calls and chat : 500000